In [1]:
# Trainer로 MLM 훈련시킨 모델에 대해 평가해보는 예시 

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, DistilBertForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# eval 말뭉치 
eval_corpus = "bongsoo/bongevalsmall"

# 기존 사전훈련된 모델
model_path = "../../data11/model/distilbert/distilbert-base-multilingual-cased/"

# 출력
OUTPATH = '../../data11/model/distilbert/dbmc-kowiki-202206-1line/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
############################################################################

device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Test-Trainer", logfilename="../../log/distilbert-MLM-Test-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbert-MLM-Test_2022-07-26.log


In [3]:
# tokeinzier 생성
tokenizer = AutoTokenizer.from_pretrained(model_path, max_len=token_max_len, do_lower_case=False)

# fast 토크너나이즈인지 확인
print(f'{model_path} is_fast:{tokenizer.is_fast}')

# 모델 로딩 further pre-training 
model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 

model.to(device)
model.eval() # 평가 

../../data11/model/distilbert/distilbert-base-multilingual-cased/ is_fast:True


DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [4]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

Using custom data configuration bongsoo--bongevalsmall-cfa82c943ea1c946
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--bongevalsmall-cfa82c943ea1c946/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})
['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 ', '게임업계 관계자는  현장 수요보다 의료진 등 특정한 누군가의 이익을 위해 게임을 중독물질  질병으로 만들려 한다는 합리적 의심이 든다 고 꼬집었다 ']


In [5]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 216 ms, sys: 105 ms, total: 320 ms
Wall time: 87.9 ms


In [6]:
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

DatasetDict({
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})
*fast_len:200, len:200
{'text': ['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 '], 'input_ids': [[101, 8909, 26444, 25242, 43022, 14646, 8900, 21611, 53639, 9489, 32537, 12310, 17196, 23635, 8844, 9554, 21611, 61844, 11261, 8922, 17730, 119432, 25387, 27506, 9944, 118859, 119403, 9855, 14040, 11513, 9952, 12310, 19905, 37321, 55910, 8922, 53726, 9937, 12965, 21711, 16139, 11018, 9651, 12310, 9521, 10739, 45893, 21711, 16139, 8888, 102055, 102], [101, 8909, 119254, 25486, 17196, 12310, 20595, 10459, 9954, 8900, 21611, 53639, 9619, 14279, 14863, 11287, 9665, 25934, 17138, 11882, 9069, 37824, 36456, 8854, 119274, 11664, 9358, 25486, 10459, 9657, 67477, 99405, 11102, 9408, 12692, 12310, 19905, 37321, 9597, 14646, 8848, 14040, 12638, 9879, 54

In [7]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:2])

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

eval_dataset_fast(MLM)=======================================
tensor([   101,   8909,  26444,    103,  43022,  14646,    103,  21611,  53639,
          9489,  32537,  12310,  17196,  23635,   8844,   9554,  21611,  61844,
           103,    103,  17730, 119432,  25387,  27506,   9944,    103, 119403,
          9855,  14040,    103,   9952,  12310,  19905,    103,    103,   8922,
         53726,   9937,  12965,  21711,  16139,  11018,    103,  12310,   9521,
         10739,  45893,  21711,  16139,   8888, 102055,    102,      0,      0,
             0,      0,      0])
{'text': '국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', 'input_ids': [101, 8909, 26444, 25242, 43022, 14646, 8900, 21611, 53639, 9489, 32537, 12310, 17196, 23635, 8844, 9554, 21611, 61844, 11261, 8922, 17730, 119432, 25387, 27506, 9944, 118859, 119403, 9855, 14040, 11513, 9952, 12310, 19905, 37321, 55910, 8922, 53726, 9937, 12965, 21711, 16139, 11018, 9651, 12310, 9521, 10739, 45893

In [8]:
# 테스트 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################

eval_steps=int(len(eval_dataset_fast["test"]) * 0.05)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)

print(f'*total_optim_steps: {len(eval_dataset_fast["test"])}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                       # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    seed=333,
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)


*total_optim_steps: 200, *eval_steps:10, *logging_steps:10
*no_cuda: False


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
# 평가 
metrics = trainer.evaluate()
logger.info('{}'.format(metrics))
            

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


2022-07-26 14:08:34,084 - distilbert-MLM-Test - INFO - {'eval_loss': 5.436809539794922, 'eval_runtime': 1.2157, 'eval_samples_per_second': 164.515, 'eval_steps_per_second': 20.564}
